In [24]:
"""
@author Dakota Rawlings
@email dakota@mrl.ucsb.edu


"""

import sqlite3
import pandas as pd
from pandas import date_range

## Importing and Formatting Data

In [25]:
#Format a dataframe after impoting from SQL
def format_df(df_in):
    """Function for formatting a dataframe from the database file with model training NDBC data
    
    :type df_in: obj (dataframe) -  Dataframe that has been uploaded from NDBC SQL database
    :rtype dataframe - formatted dataframe
    """

    df=df_in.copy(deep=True)
    df["datetime"] = pd.to_datetime(df["datetime"])
    df=df.set_index('datetime')
    df["date"] = pd.to_datetime(df["date"])
    return df

In [26]:
#Query a list of tables from the model building database
con = sqlite3.connect(r"..\NDBC_model_building_database.db")
cursor = con.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('NDBC_historical_raw_data',), ('NDBC_historical_raw_data_St#{STATIONNUMBER}',), ('NDBC_historical_raw_data_St{STATIONNUMBER}',), ('NDBC_historical_raw_data_St46054',), ('NDBC_historical_data_for_training',), ('NDBC_historical_cleaned_data',)]


In [27]:
#Query cleaned data from DB
conn = sqlite3.connect(r"..\NDBC_model_building_database.db")
df_cleaned = pd.read_sql_query("SELECT * FROM NDBC_historical_cleaned_data", conn, index_col=None)

In [28]:

conn = sqlite3.connect(r"..\NDBC_model_building_database.db")
df_training = pd.read_sql_query("SELECT * FROM NDBC_historical_data_for_training", conn, index_col=None)

In [29]:
#Format our queried data using our function defined above
df_engineered=format_df(df_cleaned)

In [30]:
#Write processed training data to new .db file for upload to google colab
conn = sqlite3.connect(r"NDBC_LSTM_building_database.db")
df_training.to_sql(name=f'NDBC_historical_data_for_training',con=conn,schema='NDBC_LSTM_building_database.db',if_exists='replace') 


3902

In [22]:
#Query cleaned data from DB
conn = sqlite3.connect(r"NDBC_LSTM_building_database.db")
df_training = pd.read_sql_query("SELECT * FROM NDBC_historical_data_for_training", conn, index_col=None)

## Basic Data EDA

In [14]:
df_engineered.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 93812 entries, 2003-06-06 14:00:00 to 2014-02-17 09:00:00
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   WDIR    93812 non-null  float64       
 1   WSPD    93812 non-null  float64       
 2   GST     93812 non-null  float64       
 3   PRES    93812 non-null  float64       
 4   ATMP    93812 non-null  float64       
 5   WTMP    93812 non-null  float64       
 6   date    93812 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(6)
memory usage: 5.7 MB


In [15]:
df_engineered.describe()

,WDIR,WSPD,GST,PRES,ATMP,WTMP
count,93812.000000,93812.000000,93812.000000,93812.000000,93812.000000,93812.000000
mean,226.870538,4.450045,5.479279,1014.965646,14.627596,15.293259
std,77.907516,2.901460,3.339500,4.085191,1.982825,2.075261
min,1.000000,0.000000,0.000000,982.800000,5.700000,10.200000
25%,206.000000,2.200000,2.900000,1012.200000,13.200000,13.600000
50%,252.000000,3.900000,4.800000,1014.600000,14.500000,15.000000
75%,276.000000,6.200000,7.400000,1017.500000,16.000000,16.800000
max,360.000000,17.300000,21.000000,1032.900000,23.800000,22.900000
